<a href="https://colab.research.google.com/github/emiliemanning/Blender-3D-Segmentation/blob/main/coco_to_mesh_disconnected_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pycocotools opencv-python-headless matplotlib scikit-image trimesh

import os
import json
from math import ceil
import numpy as np
from pycocotools import mask as maskUtils
from skimage.draw import polygon
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter
import trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 11.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving predicted_segmentations_coco (6).json to predicted_segmentations_coco (6).json


In [ ]:
from google.colab import files
import os, json

coco_json_path = list(uploaded.keys())[0]
coco_json_path = os.path.join('/content', coco_json_path)

with open(coco_json_path) as f:
    coco = json.load(f)


# first image to find height/width
if len(coco['images']) == 0:
    raise ValueError("No images found in the COCO JSON file.")

first_img = coco['images'][0]
image_height = first_img['height']
image_width = first_img['width']
num_slices = len(coco['images'])

print(f"found: height={image_height}, width={image_width}, slices={num_slices}")
output_dir = './meshes/'
os.makedirs(output_dir, exist_ok=True)

desired_blender_width = 8.0  # in Blender units
blender_unit_per_pixel = desired_blender_width / image_width
voxel_size_xy = blender_unit_per_pixel
voxel_size_z = 1 * blender_unit_per_pixel

sigma_xy = 2.5
sigma_z = 2.0

image_id_to_index = {img['id']: idx for idx, img in enumerate(sorted(coco['images'], key=lambda x: x['file_name']))}
category_id_to_name = {cat['id']: cat['name'] for cat in coco['categories']}

category_name_to_volume = {
    name: np.zeros((num_slices, image_height, image_width), dtype=np.uint8)
    for name in category_id_to_name.values()
}


found: height=350, width=500, slices=500


In [ ]:
for ann in coco['annotations']:
    slice_idx = image_id_to_index[ann['image_id']]
    category_id = ann['category_id']

    if category_id not in category_id_to_name:
        print(f"Skipping annotation ID {ann['id']} due to unknown category ID: {category_id}")
        continue

    category_name = category_id_to_name[category_id]
    segmentation = ann['segmentation']

    if isinstance(segmentation, list):  #polygon format
        for seg in segmentation:
            x = np.array(seg[0::2])
            y = np.array(seg[1::2])
            rr, cc = polygon(y, x, (image_height, image_width))
            category_name_to_volume[category_name][slice_idx, rr, cc] = 1
    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        mask = maskUtils.decode(segmentation)
        category_name_to_volume[category_name][slice_idx, mask > 0] = 1
    else:
        print(f"Unknown segmentation format for annotation ID {ann['id']}")

In [ ]:

# @title Create meshes for Blender
# meshing
z_thickness_factor = ceil(voxel_size_z / voxel_size_xy)

#calculate center offsets
x_center = image_width / 2.0
y_center = image_height / 2.0
z_center = (num_slices * z_thickness_factor) / 2.0


#meshes flat at borders
z_thickness_factor = ceil(voxel_size_z / voxel_size_xy)

# for class_name, volume in category_name_to_volume.items():
#     print(f"Meshing class: {class_name}")
#     if np.max(volume) == 0:
#         print(f"Skipping {class_name} (empty)")
#         continue
#     #scale z
#     scaled_volume = np.repeat(volume, z_thickness_factor, axis=0)
#     # gaussian
#     smoothed_volume = gaussian_filter(
#         scaled_volume.astype(np.float32),
#         sigma=(sigma_z, sigma_xy, sigma_xy)
#     )

#     # Pad volume to avoid meshing artifacts at edges
#     padded_volume = np.pad(smoothed_volume, pad_width=1, mode='constant', constant_values=0)
#     #z min
#     if np.any(volume[0, :, :]):
#         mask_at_edge = volume[0, :, :]
#         padded_volume[1, 1:-1, 1:-1][mask_at_edge > 0] = 1

#     #z max
#     if np.any(volume[-1, :, :]):
#         mask_at_edge = volume[-1, :, :]
#         padded_volume[-2, 1:-1, 1:-1][mask_at_edge > 0] = 1

#     #u min
#     if np.any(volume[:, 0, :]):
#         mask_at_edge = volume[:, 0, :]
#         padded_volume[1:-1, 1, 1:-1][mask_at_edge > 0] = 1

#     #u max
#     if np.any(volume[:, -1, :]):
#         mask_at_edge = volume[:, -1, :]
#         padded_volume[1:-1, -2, 1:-1][mask_at_edge > 0] = 1

#     #x min
#     if np.any(volume[:, :, 0]):
#         mask_at_edge = volume[:, :, 0]
#         padded_volume[1:-1, 1:-1, 1][mask_at_edge > 0] = 1

#     # x max
#     if np.any(volume[:, :, -1]):
#         mask_at_edge = volume[:, :, -1]
#         padded_volume[1:-1, 1:-1, -2][mask_at_edge > 0] = 1

#     verts, faces, normals, _ = marching_cubes(padded_volume, level=0.47)

#     # scale vertices to BLENSER units
#     new_voxel_z = voxel_size_z / z_thickness_factor
#     verts_scaled = verts * [new_voxel_z, voxel_size_xy, voxel_size_xy]

#     # center
#     z_center = padded_volume.shape[0] / 2.0
#     y_center = padded_volume.shape[1] / 2.0
#     x_center = padded_volume.shape[2] /  2.0
#     verts_scaled[:, 0] -= z_center * new_voxel_z
#     verts_scaled[:, 1] -= y_center * voxel_size_xy
#     verts_scaled[:, 2] -= x_center * voxel_size_xy

#     # reorder for Blender coords
#     mesh = trimesh.Trimesh(vertices=verts_scaled[:, [2, 1, 0]], faces=faces)
#     mesh.export(os.path.join(output_dir, f"{class_name}.ply"))

# print("MESHES EXPORTED")


Meshing class: bv
Meshing class: roi
Meshing class: tum
Meshing class: tum_extra
Skipping tum_extra (empty)
MESHES EXPORTED


In [6]:
# @title disconnected meshes for each class
for class_name, volume in category_name_to_volume.items():
    print(f"Meshing class: {class_name}")
    if np.max(volume) == 0:
        print(f"Skipping {class_name} (empty)")
        continue

    scaled_volume = np.repeat(volume, z_thickness_factor, axis=0)
    smoothed_volume = gaussian_filter(
        scaled_volume.astype(np.float32),
        sigma=(sigma_z, sigma_xy, sigma_xy)
    )
    padded_volume = np.pad(smoothed_volume, pad_width=1, mode='constant', constant_values=0)
    edges = {
        "z_min": (volume[0, :, :], (1, slice(1, -1), slice(1, -1))),
        "z_max": (volume[-1, :, :], (-2, slice(1, -1), slice(1, -1))),
        "y_min": (volume[:, 0, :], (slice(1, -1), 1, slice(1, -1))),
        "y_max": (volume[:, -1, :], (slice(1, -1), -2, slice(1, -1))),
        "x_min": (volume[:, :, 0], (slice(1, -1), slice(1, -1), 1)),
        "x_max": (volume[:, :, -1], (slice(1, -1), slice(1, -1), -2))
    }
    for mask, idx in edges.values():
        if np.any(mask):
            padded_volume[idx][mask > 0] = 1
    verts, faces, normals, _ = marching_cubes(padded_volume, level=0.47)

    new_voxel_z = voxel_size_z / z_thickness_factor
    verts_scaled = verts * [new_voxel_z, voxel_size_xy, voxel_size_xy]

    z_center = padded_volume.shape[0] / 2.0
    y_center = padded_volume.shape[1] / 2.0
    x_center = padded_volume.shape[2] / 2.0
    verts_scaled[:, 0] -= z_center * new_voxel_z
    verts_scaled[:, 1] -= y_center * voxel_size_xy
    verts_scaled[:, 2] -= x_center * voxel_size_xy

    mesh = trimesh.Trimesh(vertices=verts_scaled[:, [2, 1, 0]], faces=faces)

    # Split into connected components
    submeshes = mesh.split(only_watertight=False)

    # Export each component separately
    for i, submesh in enumerate(submeshes):
        if len(submesh.faces) == 0:
            continue  # skip empty components
        filename = os.path.join(output_dir, f"{class_name}_part{i+1}.ply")
        submesh.export(filename)
        print(f"Exported: {filename}")

print("MESHES EXPORTED")


Meshing class: bv
Exported: ./meshes/bv_part1.ply
Exported: ./meshes/bv_part2.ply
Exported: ./meshes/bv_part3.ply
Exported: ./meshes/bv_part4.ply
Exported: ./meshes/bv_part5.ply
Exported: ./meshes/bv_part6.ply
Exported: ./meshes/bv_part7.ply
Exported: ./meshes/bv_part8.ply
Exported: ./meshes/bv_part9.ply
Exported: ./meshes/bv_part10.ply
Exported: ./meshes/bv_part11.ply
Exported: ./meshes/bv_part12.ply
Exported: ./meshes/bv_part13.ply
Exported: ./meshes/bv_part14.ply
Exported: ./meshes/bv_part15.ply
Exported: ./meshes/bv_part16.ply
Exported: ./meshes/bv_part17.ply
Exported: ./meshes/bv_part18.ply
Exported: ./meshes/bv_part19.ply
Exported: ./meshes/bv_part20.ply
Exported: ./meshes/bv_part21.ply
Exported: ./meshes/bv_part22.ply
Meshing class: roi
Exported: ./meshes/roi_part1.ply
Exported: ./meshes/roi_part2.ply
Exported: ./meshes/roi_part3.ply
Exported: ./meshes/roi_part4.ply
Exported: ./meshes/roi_part5.ply
Exported: ./meshes/roi_part6.ply
Exported: ./meshes/roi_part7.ply
Exported: ./mes

In [10]:
import shutil
shutil.make_archive('meshes', 'zip', './meshes')


'/content/meshes.zip'

In [8]:
# @title Combines specific classes for tiff stack (eg: tum + roi)
combined_class_map = {
    'roi_tum': ['roi', 'tum'],
}
for new_class_name, component_names in combined_class_map.items():
    merged_volume = np.zeros_like(next(iter(category_name_to_volume.values())), dtype=np.uint8)
    for cname in component_names:
        if cname not in category_name_to_volume:
            print(f"Warning: class '{cname}' not found for combination '{new_class_name}'")
            continue
        merged_volume |= category_name_to_volume[cname]
    category_name_to_volume[new_class_name] = merged_volume


In [9]:
# @title tif stacks (different color for disconnected classes)

from scipy.ndimage import label
from skimage.color import label2rgb
from PIL import Image

tiff_output_dir = './tiff_stacks/'
os.makedirs(tiff_output_dir, exist_ok=True)

def random_colors(n):
    np.random.seed(42)
    return np.random.randint(0, 255, size=(n + 1, 3), dtype=np.uint8)

print("\nCREATING TIFF STACKS...")
for class_name, volume in category_name_to_volume.items():
    print(f"Processing class: {class_name}")
    if np.max(volume) == 0:
        print(f"Skipping {class_name} (empty)")
        continue

    # Find connected components in 3D
    structure = np.ones((3, 3, 3))
    labeled_volume, num_components = label(volume, structure=structure)

    if num_components == 0:
        print(f"No components found in {class_name}")
        continue

    # Generate random colors for each component
    colors = random_colors(num_components)
    rgb_volume = np.zeros((labeled_volume.shape[0], labeled_volume.shape[1], labeled_volume.shape[2], 3), dtype=np.uint8)
    for i in range(1, num_components + 1):
        mask = labeled_volume == i
        for c in range(3):
            rgb_volume[..., c][mask] = colors[i][c]
    rgb_volume = np.transpose(rgb_volume, (0, 1, 2, 3))  # [Z, H, W, C]
    frames = [Image.fromarray(rgb_volume[z]) for z in range(rgb_volume.shape[0])]
    tiff_path = os.path.join(tiff_output_dir, f"{class_name}.tiff")
    frames[0].save(tiff_path, save_all=True, append_images=frames[1:])
    print(f"Saved TIFF stack: {tiff_path}")

print("TIFF STACKS CREATED")




CREATING TIFF STACKS...
Processing class: bv
Saved TIFF stack: ./tiff_stacks/bv.tiff
Processing class: roi
Saved TIFF stack: ./tiff_stacks/roi.tiff
Processing class: tum
Saved TIFF stack: ./tiff_stacks/tum.tiff
Processing class: tum_extra
Skipping tum_extra (empty)
Processing class: roi_tum
Saved TIFF stack: ./tiff_stacks/roi_tum.tiff
TIFF STACKS CREATED


In [11]:
# Zip the tiff_stacks folder
shutil.make_archive('tiff_stacks', 'zip', './tiff_stacks')

print("Zipped: meshes.zip and tiff_stacks.zip")

Zipped: meshes.zip and tiff_stacks.zip
